<a href="https://colab.research.google.com/github/workhardzy/K6312/blob/main/topic_modelling_COVID_news_ipynb%EF%BC%88for_topic_modeling%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the libraries needed

In [ ]:
#note you need to have it on your own google drive,for those who are sharing a drive, an approach is to add shortcut to your drive. 
#See https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab
#put in your authorisation code that is linked to the google account you are linking to and press enter
from google.colab import drive
drive.mount("/content/gdrive",force_remount=True)
!ls "/content/gdrive/"

Mounted at /content/gdrive
'My Drive'


In [ ]:
!pip install pyLDAvis

In [ ]:
#open file
import os
#import display
from IPython.display import display

import pandas as pd
import re, pickle, os
import datetime 
import nltk
from nltk.util import ngrams
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords, wordnet 
from collections import Counter 
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
from gensim.corpora import MmCorpus
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim

import matplotlib.pyplot as plt
%matplotlib inline
import math
import ast

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Utils

In [ ]:
def save_checkpoint(df,filepath):
    df.to_pickle(filepath)
    print('saved dataframe at {}'.format(filepath)) 


In [ ]:
#these are the functions to train the model

def save_print_to_file(outfile, msg):
    with open(outfile, 'w') as fp:
        print(msg, file=fp)  

def get_word_count(tweets_text, num_gram):
    '''
    Get common word counts
    '''
    n_grams = list(ngrams(tweets_text, num_gram))
    
    #stop_ngrams = ['local news']
    #n_grams = [i for i in n_grams if i not in stop_ngrams]
    
    common_words = Counter(n_grams).most_common()
    word_count = pd.DataFrame(data = common_words, 
                              columns=['word','frequency']) 
    # Convert list to string
    word_count['word'] = word_count['word'].apply(' '.join)
    # Plot word count graph
    word_count.head(20).sort_values('frequency').plot.barh(
            x='word', y='frequency', title='Word Frequency',figsize=(19,10))
    plt.savefig(WORD_COUNT_FILE)
    print ('Word count saved\n')
    plt.close('all')
    
    return word_count

def word_grams(words, min=1, max=2):
    '''
    Build ngrams word list
    '''
    word_list = []
    for n in range(min, max):
        for ngram in ngrams(words, n):
            #print(ngram)
            word_list.append(' '.join(str(i) for i in ngram)) 
    return word_list

def train_lda_model(token_tweets):
    print('Start LDA model training ...\n')    
    # Build dictionary
    print('building dictionary')
    tweets_dict = corpora.Dictionary(token_tweets)
    # Remove words that occur less than 10 documents, # or more than 50% of the doc
    tweets_dict.filter_extremes(no_below=10, no_above=0.5)                       ########调
    # Transform doc to a vectorized form by computing frequency of each word
    bow_corpus = [tweets_dict.doc2bow(doc) for doc in token_tweets]
    # Save corpus and dictionary to file
    MmCorpus.serialize(CORPUS_FILE, bow_corpus)
    tweets_dict.save(DICT_FILE)
    print('saved corpus and dictionary to file')
    
    # Create tf-idf model and then apply transformation to the entire corpus
    print('create tf-idf model')
    tfidf = models.TfidfModel(bow_corpus)                                       ########调 (optional)
    tfidf_corpus = tfidf[bow_corpus]                                            ########调 (optional)
    
    print('training lda')
    # Train LDA model #this is the time bottleneck
    lda_model = models.ldamodel.LdaModel(corpus=tfidf_corpus,                   ########调 (optional)
                                         num_topics=NUM_TOPICS, 
                                         id2word=tweets_dict, 
                                         passes=NUM_PASSES, 
                                         alpha=ALPHA, 
                                         eta=ETA,
                                         random_state=42)
    # Save LDA model to file
    lda_model.save(LDA_MODEL_FILE)
    print ('LDA model saved\n')
      
    # Save all generated topics to a file
    msg = ''
    for idx, topic in lda_model.print_topics(-1):
        msg += 'Topic: {} \nWords: {}\n'.format(idx, topic)    
    save_print_to_file(LDA_TOPICS_FILE, msg)
    
    # Evaluate LDA model performance
    eval_lda (lda_model, tfidf_corpus, tweets_dict, token_tweets)    
    # Visualize topics
    vis_topics(lda_model, tfidf_corpus, tweets_dict)
        
    return lda_model

def eval_lda (lda_model, corpus, dict, token_text):
    
    # Compute Perplexity: a measure of how good the model is. lower the better.
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
    
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=token_text, 
                                         dictionary=dict, coherence='c_v')   
    print('\nCoherence: ', coherence_model_lda.get_coherence())

def vis_topics(lda_model, corpus, dict):
    '''
    Plot generated topics on an interactive graph
    '''
    lda_data =  pyLDAvis.gensim.prepare(lda_model, corpus, dict, mds='mmds')
    pyLDAvis.display(lda_data)
    pyLDAvis.save_html(lda_data, TOPIC_VIS_FILE)
    print ('Topic visual saved\n')

def wordcloud(word_count_df):
    '''
    Create word cloud image
    '''
    # Convert DataFrame to Map so that word cloud can be generated from freq
    word_count_dict = {}
    for w, f in word_count_df.values:
        word_count_dict[w] = f
    # Generate word cloud 
    wordcloud = WordCloud(max_words=300, width=1400, height=900, 
                          random_state=12, contour_width=3, 
                          contour_color='firebrick')
    wordcloud.generate_from_frequencies(word_count_dict)
    plt.figure(figsize=(10,10), facecolor='k')
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    # Save the word cloud image
    wordcloud.to_file(WORDCLOUD_FILE) 
    print ('Word cloud saved\n')
    plt.close('all')
    
    return wordcloud

In [ ]:
'''
def clean_topics(df):
    print('CLEANING...')
    cleaned_tweets_df = preprocess_tweets(df,'tweet_text') ########调
    cleaned_tweets_df.to_pickle('/content/gdrive/My Drive/group/data/preprocessed_news_headlines.h5') ########调
    return cleaned_tweets_df
'''    
def generate_ngrams(cleaned_tweets_df):
    # Convert series to list for word count
    print('creating tweets_text corpus')
    tweets_text = [word for one_tweet in cleaned_tweets_df['token'] for word in one_tweet]
    
    print('getting ngram word count')
    # Get common ngrams word count
    word_count_df = get_word_count(tweets_text, num_gram=NUM_GRAMS)
    
    # Generate word cloud
    tweets_wordcloud = wordcloud(word_count_df)  

    # Generate ngram tokens
    cleaned_tweets_df['ngram_token'] = [word_grams(x, NUM_GRAMS, NUM_GRAMS+1) for x in cleaned_tweets_df['token']]
    
    
    return cleaned_tweets_df

    
def trainingLDA(ngrams):
    print('TRAINING...')
    # Train LDA model and visualize generated topics
    lda_model = train_lda_model(ngrams)

    print('DONE!')
    
    return lda_model

Here we start training the model and the data

In [ ]:
#path variables
mth = 9   #to change the month                                                  ########调
FIGURE_PATH = r'gdrive/My Drive/group/figures/'                          ########调 #need to create the folder
DATA_PATH = r'gdrive/My Drive/group/data/'                               ########调 #need to create the folder
MODEL_PATH = r'gdrive/My Drive/group/models/'                            ########调 #need to create the folder
WORDCLOUD_FILE = FIGURE_PATH + 'wordcloud{}.png'.format(mth)
WORD_COUNT_FILE = FIGURE_PATH + 'commond_words_freq{}.png'.format(mth)
TOPIC_VIS_FILE = FIGURE_PATH + 'lda{}.html'.format(mth)
ORIG_TWEET_FILE = DATA_PATH + 'all_tweets'                                      ########换名字 
CLEANED_TWEET_FILE = DATA_PATH + 'tweets_cleaned_df'                            ########换名字 
CORPUS_FILE = MODEL_PATH + 'clean_tweets_corpu{}s.mm'.format(mth)                             ########换名字 
DICT_FILE = MODEL_PATH + 'clean_tweet{}s.dict'.format(mth)                                    ########换名字 
LDA_MODEL_FILE = MODEL_PATH + 'tweets_lda{}.model'.format(mth)                                ########换名字 
LDA_TOPICS_FILE = MODEL_PATH + 'tweets_lda_topics{}.txt'.format(mth)                          ########换名字 


#tunable parameters
NUM_GRAMS = 2 #ngrams                                                                                                                    ########调 (optional)
NUM_TOPICS = 5 #guesstimate  #要用分析                                                                                                   ########调 
NUM_PASSES = 50 #Number of training passes/iterations over all tweets   #越高越久，但应该会准                                              ########调 (optional)         
ALPHA = 'auto'  #the lower alpha is, the more likely that a tweet may contain mixture of just a few of the topics 1/auto is normal
ETA = 'auto' # Word-Topic Density.The lower eta is, the more likely that a topic may contain a mixture of just a few of the words

#system variables
WORDCLOUD_FILE = FIGURE_PATH + 'wordcloud{}.png'.format(mth)                                  ########换名字 
WORD_COUNT_FILE = FIGURE_PATH + 'common_words_freq{}.png'.format(mth)                         ########换名字   
TOPIC_VIS_FILE = FIGURE_PATH + 'lda{}.html'.format(mth)                                       ########换名字 
CORPUS_FILE = MODEL_PATH + 'clean_tweets_corpus{}.mm'.format(mth)                             ########换名字 
DICT_FILE = MODEL_PATH + 'clean_tweets{}.dict'.format(mth)                                    ########换名字 
LDA_MODEL_FILE = MODEL_PATH + 'tweets_lda{}.model'.format(mth)                                ########换名字 



In [ ]:
df = pd.read_pickle('/content/gdrive/My Drive/group/data/preprocessed_news_headlines{}.h5'.format(mth)) ########调
df

,title,date,topic_area,token,ngram_token,month
273937,Samsung eyes new opportunities as pandemic ignites demand for home appliances,2020-09-01,business,"[samsung, eye, new, opportunity, pandemic, ignites, demand, home, appliance]","[samsung eye, eye new, new opportunity, opportunity pandemic, pandemic ignites, ignites demand, demand home, home appliance]",9
273938,"Horse racing: Tiz the Law arrives, Art Collector exits at Churchill Downs",2020-09-01,business,"[horse, race, tiz, law, arrives, art, collector, exit, churchill]","[horse race, race tiz, tiz law, law arrives, arrives art, art collector, collector exit, exit churchill]",9
273939,US Open 2020: Osaka passes Doi test and Pliskova advances but Gauff falls,2020-09-01,business,"[u, open, osaka, pass, doi, test, pliskova, advance, gauff, fall]","[u open, open osaka, osaka pass, pass doi, doi test, test pliskova, pliskova advance, advance gauff, gauff fall]",9
273940,The future of sports TV coverage? Eurosport unveils mixed-reality 'Cube' studio for US Open,2020-09-01,business,"[future, sport, tv, coverage, eurosport, unveils, mixed, reality, cube, studio, u, open]","[future sport, sport tv, tv coverage, coverage eurosport, eurosport unveils, unveils mixed, mixed reality, reality cube, cube studio, studio u, u open]",9
273941,U.S. employment projected to increase six million from 2019 to 2029: Labor Department,2020-09-01,business,"[u, employment, project, increase, six, million, labor, department]","[u employment, employment project, project increase, increase six, six million, million labor, labor department]",9
...,...,...,...,...,...,...
290474,"Trump signs new, expanded executive order to lower U.S. drug prices",2020-09-14,business,"[trump, sign, new, expand, executive, order, lower, u, drug, price]","[trump sign, sign new, new expand, expand executive, executive order, order lower, lower u, u drug, drug price]",9
290475,Nuggets force Game 7 and D'Antoni leaves HOU,2020-09-14,business,"[nugget, force, game, antoni, leaf, hou]","[nugget force, force game, game antoni, antoni leaf, leaf hou]",9
290476,Health Official Out To Manipulate CDC Reports Has Deep Russian Ties,2020-09-14,business,"[health, official, manipulate, cdc, report, deep, russian, tie]","[health official, official manipulate, manipulate cdc, cdc report, report deep, deep russian, russian tie]",9
290477,Global Markets: Asian shares buoyed by coronavirus vaccine hopes,2020-09-14,business,"[global, market, asian, share, buoyed, coronavirus, vaccine, hope]","[global market, market asian, asian share, share buoyed, buoyed coronavirus, coronavirus vaccine, vaccine hope]",9


#training the model

can skip if not needed to train

In [ ]:
#This is the part where we suppress warnings as it is occupying too much memory
import warnings
warnings.filterwarnings('ignore')

tokens = df['ngram_token']

%time LDAmodel = trainingLDA(tokens) #training done, #this portion takes a lot of time 

'''
CPU times: user 54min 36s, sys: 15.8 s, total: 54min 52s
Wall time: 55min 8s

Perplexity, the lower the better -93.7418325786182

Coherence, the higher the better 0.6607926560682633
'''
'''
Perplexity:  -5.348532548095669

Coherence:  0.5406701187135824
'''

TRAINING...
Start LDA model training ...

building dictionary
saved corpus and dictionary to file
create tf-idf model
training lda
LDA model saved


Perplexity:  -6.608657366805812

Coherence:  0.757562918211186
Topic visual saved

DONE!
CPU times: user 1min 11s, sys: 124 ms, total: 1min 11s
Wall time: 1min 13s


'\nPerplexity:  -5.348532548095669\n\nCoherence:  0.5406701187135824\n'

#to analyse

In [ ]:
#showing the topics and the keywords
LDA_MODEL_FILE = MODEL_PATH + 'tweets_lda{}.model'.format(mth) 
ldaModel = models.ldamodel.LdaModel.load(LDA_MODEL_FILE)
for j in ldaModel.print_topics(-1):
    print(j)

(0, '0.033*"donald trump" + 0.033*"covid case" + 0.029*"prime minister" + 0.028*"news guardian" + 0.025*"stock market" + 0.024*"researchandmarkets com" + 0.024*"win u" + 0.021*"amid pandemic" + 0.020*"season opener" + 0.018*"face mask"')
(1, '0.049*"new york" + 0.024*"bob woodward" + 0.023*"south korea" + 0.022*"film festival" + 0.022*"u k" + 0.021*"coronavirus pandemic" + 0.018*"first time" + 0.016*"york city" + 0.016*"u election" + 0.015*"asia today"')
(2, '0.055*"coronavirus vaccine" + 0.034*"wall street" + 0.022*"report marketscreener" + 0.022*"labor day" + 0.019*"covid pandemic" + 0.019*"global market" + 0.019*"update marketscreener" + 0.018*"recovery covid" + 0.017*"white house" + 0.017*"japan pm"')
(3, '0.075*"covid vaccine" + 0.046*"market growth" + 0.043*"growth technavio" + 0.043*"boost market" + 0.041*"vaccine trial" + 0.035*"indoor rally" + 0.023*"premier league" + 0.021*"roadmap recovery" + 0.021*"new zealand" + 0.020*"boris johnson"')
(4, '0.098*"u open" + 0.041*"covid te

In [ ]:
#This is the part where we suppress warnings as it is occupying too much memory
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from tqdm import tqdm

#creating a dataframe for creating topic 
#load model
#load dataframe

def create_topic_df(df):
    CORPUS_FILE = MODEL_PATH + 'clean_tweets_corpus{}.mm'.format(mth)                      ########换名字 
    DICT_FILE = MODEL_PATH + 'clean_tweets{}.dict'.format(mth)                             ########换名字 
    LDA_MODEL_FILE = MODEL_PATH + 'tweets_lda{}.model'.format(mth)                         ########换名字 

    print(LDA_MODEL_FILE)
    ldaModel = models.ldamodel.LdaModel.load(LDA_MODEL_FILE)
    #load corpus
    print(CORPUS_FILE)
    corpus = MmCorpus(CORPUS_FILE)
    
    d_lookup = pd.DataFrame(ldaModel.print_topics(-1))
    
    topic_LDA_df = pd.DataFrame()
    
    for j in tqdm(range(len(corpus))):   
        tldf = pd.DataFrame(ldaModel[corpus[j]]).transpose().drop(0)
        a = tldf
        tldf['max_pred_value']= a.max(axis=1)
        tldf['pred_topic'] = a.idxmax(axis=1)
        tldf['topic_label'] = d_lookup[d_lookup[0]==tldf['pred_topic'].iloc[0]][1].iloc[0]
        topic_LDA_df = topic_LDA_df.append(tldf)

    topic_LDA_df.reset_index(drop=True,inplace=True)

    topic_LDA_df = topic_LDA_df.add_prefix('topic_')

    return topic_LDA_df


In [ ]:
CORPUS_FILE = MODEL_PATH + 'clean_tweets_corpus{}.mm'.format(mth)                      ########换名字 
DICT_FILE = MODEL_PATH + 'clean_tweets{}.dict'.format(mth)                             ########换名字 
LDA_MODEL_FILE = MODEL_PATH + 'tweets_lda{}.model'.format(mth)                         ########换名字 

print(LDA_MODEL_FILE)
ldaModel = models.ldamodel.LdaModel.load(LDA_MODEL_FILE)
#load corpus
print(CORPUS_FILE)
corpus = MmCorpus(CORPUS_FILE)

d_lookup = pd.DataFrame(ldaModel.print_topics(-1))

topic_LDA_df = pd.DataFrame()

for j in tqdm(range(10)):   
    tldf = pd.DataFrame(ldaModel[corpus[j]]).transpose().drop(0)
    a = tldf
    tldf['max_pred_value']= a.max(axis=1)
    tldf['pred_topic'] = a.idxmax(axis=1)
    tldf['topic_label'] = d_lookup[d_lookup[0]==tldf['pred_topic'].iloc[0]][1].iloc[0]
    topic_LDA_df = topic_LDA_df.append(tldf)

topic_LDA_df.reset_index(drop=True,inplace=True)

topic_LDA_df = topic_LDA_df.add_prefix('topic_')
topic_LDA_df

100%|██████████| 10/10 [00:00<00:00, 140.83it/s]

gdrive/My Drive/group/models/tweets_lda9.model
gdrive/My Drive/group/models/clean_tweets_corpus9.mm


,topic_0,topic_1,topic_2,topic_3,topic_4,topic_max_pred_value,topic_pred_topic,topic_topic_label
0,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
1,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
2,0.153535,0.138480,0.162979,0.141370,0.403637,0.403637,4,"0.098*""u open"" + 0.041*""covid test"" + 0.032*""coronavirus case"" + 0.030*""test positive"" + 0.026*""first half"" + 0.025*""grand slam"" + 0.022*""rule six"" + 0.019*""oil price"" + 0.019*""hong kong"" + 0.018*""opinion cnn"""
3,0.153535,0.138480,0.162979,0.141370,0.403637,0.403637,4,"0.098*""u open"" + 0.041*""covid test"" + 0.032*""coronavirus case"" + 0.030*""test positive"" + 0.026*""first half"" + 0.025*""grand slam"" + 0.022*""rule six"" + 0.019*""oil price"" + 0.019*""hong kong"" + 0.018*""opinion cnn"""
4,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
5,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
6,0.153535,0.138480,0.162979,0.399626,0.145381,0.399626,3,"0.075*""covid vaccine"" + 0.046*""market growth"" + 0.043*""growth technavio"" + 0.043*""boost market"" + 0.041*""vaccine trial"" + 0.035*""indoor rally"" + 0.023*""premier league"" + 0.021*""roadmap recovery"" + 0.021*""new zealand"" + 0.020*""boris johnson"""
7,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
8,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
9,0.122022,0.110057,0.540026,0.112354,0.115541,0.540026,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""


In [ ]:
#execution
dd = create_topic_df(df) #this takes a really long time. 1:19:37. 1 Looking to optimise this for future work
display(dd)
save_checkpoint(dd,'/content/gdrive/My Drive/group/data/topic_modelled_news_headlines{}.h5'.format(mth)) ########调


  0%|          | 16/16301 [00:00<01:44, 156.57it/s]

gdrive/My Drive/group/models/tweets_lda9.model
gdrive/My Drive/group/models/clean_tweets_corpus9.mm


100%|██████████| 16301/16301 [01:43<00:00, 158.12it/s]


,topic_0,topic_1,topic_2,topic_3,topic_4,topic_max_pred_value,topic_pred_topic,topic_topic_label
0,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
1,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
2,0.153535,0.138480,0.162979,0.141370,0.403637,0.403637,4,"0.098*""u open"" + 0.041*""covid test"" + 0.032*""coronavirus case"" + 0.030*""test positive"" + 0.026*""first half"" + 0.025*""grand slam"" + 0.022*""rule six"" + 0.019*""oil price"" + 0.019*""hong kong"" + 0.018*""opinion cnn"""
3,0.153535,0.138480,0.162979,0.141370,0.403637,0.403637,4,"0.098*""u open"" + 0.041*""covid test"" + 0.032*""coronavirus case"" + 0.030*""test positive"" + 0.026*""first half"" + 0.025*""grand slam"" + 0.022*""rule six"" + 0.019*""oil price"" + 0.019*""hong kong"" + 0.018*""opinion cnn"""
4,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
...,...,...,...,...,...,...,...,...
16296,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
16297,0.206992,0.186695,0.219724,0.190591,0.195999,0.219724,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
16298,0.122022,0.110057,0.334777,0.317603,0.115541,0.334777,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""
16299,0.101242,0.091314,0.448062,0.263516,0.095865,0.448062,2,"0.055*""coronavirus vaccine"" + 0.034*""wall street"" + 0.022*""report marketscreener"" + 0.022*""labor day"" + 0.019*""covid pandemic"" + 0.019*""global market"" + 0.019*""update marketscreener"" + 0.018*""recovery covid"" + 0.017*""white house"" + 0.017*""japan pm"""


saved dataframe at /content/gdrive/My Drive/group/data/topic_modelled_news_headlines9.h5


In [ ]:
pred = pd.read_pickle('/content/gdrive/My Drive/group/data/topic_modelled_news_headlines{}.h5'.format(mth)) ########调
df2 = df.join(pred)
df2

,title,date,topic_area,token,ngram_token,month,topic_0,topic_1,topic_2,topic_3,topic_4,topic_max_pred_value,topic_pred_topic,topic_topic_label
273937,Samsung eyes new opportunities as pandemic ignites demand for home appliances,2020-09-01,business,"[samsung, eye, new, opportunity, pandemic, ignites, demand, home, appliance]","[samsung eye, eye new, new opportunity, opportunity pandemic, pandemic ignites, ignites demand, demand home, home appliance]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273938,"Horse racing: Tiz the Law arrives, Art Collector exits at Churchill Downs",2020-09-01,business,"[horse, race, tiz, law, arrives, art, collector, exit, churchill]","[horse race, race tiz, tiz law, law arrives, arrives art, art collector, collector exit, exit churchill]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273939,US Open 2020: Osaka passes Doi test and Pliskova advances but Gauff falls,2020-09-01,business,"[u, open, osaka, pass, doi, test, pliskova, advance, gauff, fall]","[u open, open osaka, osaka pass, pass doi, doi test, test pliskova, pliskova advance, advance gauff, gauff fall]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273940,The future of sports TV coverage? Eurosport unveils mixed-reality 'Cube' studio for US Open,2020-09-01,business,"[future, sport, tv, coverage, eurosport, unveils, mixed, reality, cube, studio, u, open]","[future sport, sport tv, tv coverage, coverage eurosport, eurosport unveils, unveils mixed, mixed reality, reality cube, cube studio, studio u, u open]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273941,U.S. employment projected to increase six million from 2019 to 2029: Labor Department,2020-09-01,business,"[u, employment, project, increase, six, million, labor, department]","[u employment, employment project, project increase, increase six, six million, million labor, labor department]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290474,"Trump signs new, expanded executive order to lower U.S. drug prices",2020-09-14,business,"[trump, sign, new, expand, executive, order, lower, u, drug, price]","[trump sign, sign new, new expand, expand executive, executive order, order lower, lower u, u drug, drug price]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290475,Nuggets force Game 7 and D'Antoni leaves HOU,2020-09-14,business,"[nugget, force, game, antoni, leaf, hou]","[nugget force, force game, game antoni, antoni leaf, leaf hou]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290476,Health Official Out To Manipulate CDC Reports Has Deep Russian Ties,2020-09-14,business,"[health, official, manipulate, cdc, report, deep, russian, tie]","[health official, official manipulate, manipulate cdc, cdc report, report deep, deep russian, russian tie]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290477,Global Markets: Asian shares buoyed by coronavirus vaccine hopes,2020-09-14,business,"[global, market, asian, share, buoyed, coronavirus, vaccine, hope]","[global market, market asian, asian share, share buoyed, buoyed coronavirus, coronavirus vaccine, vaccine hope]",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_most_rep = df2

#compare 2 lists of tokens for matching comparison, essentially, it tells us if the tokens of this news headlines are the keywords used for the topic
df_most_rep['topic_topic_label'] = df_most_rep['topic_topic_label'].apply(lambda x: re.findall(r'"(.*?)"', x))

shared_tokens = []
for j in tqdm(range(len(df_most_rep))):
    line = df_most_rep.iloc[j]
    tokensA = line['topic_topic_label']
    tokensB = line['ngram_token']
    tokensC = list(set(tokensA).intersection(set(tokensB)))
    shared_tokens.append(tokensC)

df_most_rep['common tokens'] = pd.Series(shared_tokens)
df_most_rep['common tokens count'] = df_most_rep['common tokens'].apply(lambda x: len(x))

save_checkpoint(df_most_rep,'/content/gdrive/My Drive/group/data/topic_modelled_news_headlines.h5') ########调



TypeError: ignored

In [ ]:
#junk code used for finding the top n tweets
'''
for i in range(1,13):
    print('Month '+ str(i))
    df_most_rep = pd.read_pickle('gdrive/My Drive/IO - twitter datasets/russia/topicmodellingIRA_classified_{}.h5'.format(i))

    df_most_rep = df_most_rep[df_most_rep['tweet_year'] == 2016]

        
    #find top 10 tweets for each category
    df_most_rep = df_most_rep[df_most_rep['common tokens count']>0] #filter for shared words with topics
    

    sent_topics_outdf_grpd = df_most_rep[['tweet_text','user_screen_name', 'retweet_count','ngram_token','topic_pred_topic',
                 'topic_max_pred_value', 'topic_topic_label','common tokens','common tokens count']].groupby('topic_pred_topic')

    sent_topics_sorteddf = pd.DataFrame()
    for k, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, 
                                                grp.sort_values(['retweet_count'], ascending=[0]).head(5)], 
                                                axis=0)

    sent_topics_sorteddf.reset_index(drop=True, inplace=True)
    sent_topics_sorteddf.to_excel('gdrive/My Drive/IO - twitter datasets/russia/top_tweets_IRA_2016_month{}.xlsx'.format(i))
    sent_topics_sorteddf.to_pickle('gdrive/My Drive/IO - twitter datasets/russia/sortedtoptentopicmodellingIRA_classified_{}.h5'.format(i))

SyntaxError: ignored

# To do analysis and results reporting

In [ ]:
pd.set_option('display.max_colwidth', -1)

df_topic_sents_keywords = pd.read_pickle('/content/gdrive/My Drive/group/data/topic_modelled_news_headlines.h5') ########调

topic_num_keywords = df_topic_sents_keywords[['topic_pred_topic', 'topic_topic_label']]

topic_num_keywords['topic_topic_label'] = topic_num_keywords['topic_topic_label'].apply(lambda x: str(x))
topic_num_keywords = topic_num_keywords.groupby(['topic_pred_topic', 'topic_topic_label']).count().reset_index()    


topic_counts = df_topic_sents_keywords['topic_pred_topic'].value_counts().reset_index()
topic_counts.columns = ['topic_pred_topic','counts']
topic_contribution = topic_counts['counts'].apply(lambda x: round(x/topic_counts.counts.sum(), 3)*100)
dfa = pd.concat([topic_counts,topic_contribution],axis=1)


df_dominant_topics = pd.merge(topic_num_keywords,dfa,on='topic_pred_topic')
df_dominant_topics.columns = ['Dominant Topic', 'Topic Keywords', 'Number of Documents', 'Percentage of Tweets']

df_dominant_topics['Dominant Topic'] = df_dominant_topics.index

df_dominant_topics['Topic Keywords'] = df_dominant_topics['Topic Keywords'].apply(lambda x: ast.literal_eval(x))
df_dominant_topics['Topic Keywords'] = df_dominant_topics['Topic Keywords'].apply(lambda x: ', '.join([str(elem) for elem in x]) )
df_dominant_topics['Topic Keywords'] = df_dominant_topics['Dominant Topic'].apply(lambda x: 'Topic ' + str(x) + ': ') + df_dominant_topics['Topic Keywords']

display(df_dominant_topics)

,Dominant Topic,Topic Keywords,Number of Documents,Percentage of Tweets
0,0,"Topic 0: coronavirus vaccine, coronavirus concern, due coronavirus, case coronavirus, bbc news, china travel, asia stock, scientific american, coronavirus test, treasury yield",86,4.4
1,1,"Topic 1: coronavirus fear, virus outbreak, novel coronavirus, china economy, new virus, deadly coronavirus, public health, health emergency, fall coronavirus, asian market",113,5.8
2,2,"Topic 2: co uk, express co, news express, world news, coronavirus spread, new coronavirus, coronavirus latest, uk news, deadly virus, science news",913,46.7
3,3,"Topic 3: wuhan coronavirus, coronavirus marketwatch, q earnings, fourth quarter, first case, case confirm, confirms first, motley fool, earnings call, oil price",127,6.5
4,4,"Topic 4: news guardian, world news, death toll, coronavirus death, football sport, sport express, transfer news, toll rise, european stock, spread cnn",175,9.0
5,5,"Topic 5: china virus, virus fear, u stock, update oil, virus toll, china flight, declares coronavirus, fear grow, public health, emerge market",126,6.5
6,6,"Topic 6: coronavirus outbreak, amid coronavirus, coronavirus could, coronavirus case, say coronavirus, south china, global health, china morning, morning post, outbreak china",159,8.1
7,7,"Topic 7: global market, travel express, news travel, coronavirus cnn, travel news, travel china, u case, emerge market, stock fall, co uk",82,4.2
8,8,"Topic 8: china coronavirus, virus spread, coronavirus china, coronavirus u, new year, lunar new, wuhan virus, spread coronavirus, coronavirus outbreak, news guardian",98,5.0
9,9,"Topic 9: stock market, hong kong, outbreak cnn, update marketscreener, live update, market open, coronavirus uk, coronavirus outbreak, coronavirus hit, co uk",74,3.8


In [ ]:
dis = pd.read_pickle('/content/gdrive/My Drive/group/data/topic_modelled_news_headlines.h5')              ########调
topicN = 3                                                                                                                      ########调
dis[dis['topic_pred_topic']==topicN]                                                                                             


,title,date,topic_area,token,ngram_token,month,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_max_pred_value,topic_pred_topic,topic_topic_label,common tokens,common tokens count
1,"Tesla (TSLA) Breaks Shipment Record, Beats Estimates for Fourth Quarter Vehicles Shipped",2020-01-03,business,"[tesla, tsla, break, shipment, record, beat, estimate, fourth, quarter, vehicle, ship]","[tesla tsla, tsla break, break shipment, shipment record, record beat, beat estimate, estimate fourth, fourth quarter, quarter vehicle, vehicle ship]",1,0.046584,0.049803,0.056376,0.553383,0.050380,0.048434,0.052530,0.046979,0.049592,0.045942,0.553383,3,"[wuhan coronavirus, coronavirus marketwatch, q earnings, fourth quarter, first case, case confirm, confirms first, motley fool, earnings call, oil price]",[fourth quarter],1
44,Japan confirms first case of new China coronavirus strain | World news | The Guardian,2020-01-16,general,"[japan, confirms, first, case, new, china, coronavirus, strain, world, news, guardian]","[japan confirms, confirms first, first case, case new, new china, china coronavirus, coronavirus strain, strain world, world news, news guardian]",1,0.015442,0.016509,0.018693,0.351987,0.327934,0.016055,0.017414,0.015573,0.205164,0.015229,0.351987,3,"[wuhan coronavirus, coronavirus marketwatch, q earnings, fourth quarter, first case, case confirm, confirms first, motley fool, earnings call, oil price]","[first case, confirms first]",2
93,New China virus: US announces first case - BBC News,2020-01-21,general,"[new, china, virus, u, announces, first, case, bbc, news]","[new china, china virus, virus u, u announces, announces first, first case, case bbc, bbc news]",1,0.272719,0.024797,0.028071,0.277426,0.025086,0.274786,0.026155,0.023391,0.024693,0.022875,0.277426,3,"[wuhan coronavirus, coronavirus marketwatch, q earnings, fourth quarter, first case, case confirm, confirms first, motley fool, earnings call, oil price]",[first case],1
116,Dow tumbles on first case of coronavirus in the United States - CNN,2020-01-21,general,"[dow, tumble, first, case, coronavirus, united, state, cnn]","[dow tumble, tumble first, first case, case coronavirus, coronavirus united, united state, state cnn]",1,0.236655,0.033109,0.037479,0.497397,0.033493,0.032199,0.034922,0.031232,0.032969,0.030543,0.497397,3,"[wuhan coronavirus, coronavirus marketwatch, q earnings, fourth quarter, first case, case confirm, confirms first, motley fool, earnings call, oil price]",[first case],1
133,Wuhan coronavirus case confirmed in Washington state - CNN,2020-01-21,general,"[wuhan, coronavirus, case, confirm, washington, state, cnn]","[wuhan coronavirus, coronavirus case, case confirm, confirm washington, washington state, state cnn]",1,0.023195,0.024798,0.028072,0.777610,0.025085,0.024116,0.026166,0.023391,0.024692,0.022875,0.777610,3,"[wuhan coronavirus, coronavirus marketwatch, q earnings, fourth quarter, first case, case confirm, confirms first, motley fool, earnings call, oil price]","[wuhan coronavirus, case confirm]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,Visa Inc (V) Q1 2020 Earnings Call Transcript | The Motley Fool,2020-01-31,business,"[visa, inc, v, q, earnings, call, transcript, motley, fool]","[visa inc, inc v, v q, q earnings, earnings call, call transcript, transcript motley, motley fool]",1,0.023195,0.024797,0.028070,0.777623,0.025085,0.024116,0.026155,0.023391,0.024692,0.022875,0.777623,3,"[wuhan coronavirus, coronavirus marketwatch, q earnings, fourth quarter, first case, case confirm, confirms first, motley fool, earnings call, oil price]","[motley fool, earnings call, q earnings]",3
1740,Illinois Tool Works Inc (ITW) Q4 2019 Earnings Call Transcript | The Motley Fool,2020-01-31,business,"[illinois, tool, work, inc, itw, q, earnings, call, transcript, motley, fool]","[illinois tool, tool work, work inc, inc itw, itw q, q earnings, earnings call, call transcript, trans

In [ ]:
#sort by predicted value
topicN = 16
dis = pd.read_pickle('/content/gdrive/My Drive/group/data/topic_modelled_news_headlines.h5')   ########调
dis[dis['topic_pred_topic']==topicN].sort_values('topic_max_pred_value')


,title,date,topic_area,token,ngram_token,month,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_max_pred_value,topic_pred_topic,topic_topic_label,common tokens,common tokens count


In [ ]:
#export
dis.to_csv('/content/gdrive/My Drive/group/data/topics.csv',encoding='utf-8') ########调